In [1]:
import geopandas as gpd
from libpysal import graph
import shapely
import pandas as pd
import numpy as np

import lonboard
from sidecar import Sidecar

In [2]:
buildings = gpd.read_parquet("/data/uscuni-ulce/processed_data/simplified_buildings/buildings_69333.parquet").reset_index()
prg_estates = gpd.read_file("/data/uscuni-ulce/additional_data/housing_estates_locations/prague/sidliste.shp").to_crs(buildings.crs)

In [3]:
buildings.shape

(457127, 6)

In [4]:
buildings['area'] = buildings.area
buildings['perimeter'] = buildings.length

In [5]:
_, blg_ix = buildings.sindex.query(prg_estates.geometry, predicate="intersects")
relevant = pd.Series(0, index=buildings.index, dtype=bool)
relevant.iloc[np.unique(blg_ix)] = True

In [6]:
min_area = 180
min_perimeter = 55

In [7]:
contig = graph.Graph.build_fuzzy_contiguity(buildings, buffer=1e-3)

In [8]:
mask = (buildings['area'] > min_area) & (buildings['perimeter'] > min_perimeter) & relevant

In [9]:
masked = buildings[mask].copy()
masked.shape

(7515, 8)

In [10]:
masked

,index,currentUse,buildingNature,heightAboveGround,iid,geometry,area,perimeter
108089,108089,None,None,NaN,82152767,"POLYGON ((4637654.797 3011971.617, 4637654.983...",596.370442,125.505235
108104,108104,None,None,NaN,82152782,"POLYGON ((4637288.535 3011544.855, 4637289.08 ...",208.257335,60.404515
108110,108110,None,None,NaN,82152788,"POLYGON ((4637162.143 3011664.644, 4637162.552...",2060.249626,212.677229
108116,108116,None,None,NaN,82152794,"POLYGON ((4637800.893 3011594.356, 4637812.655...",540.769614,107.235078
108118,108118,None,None,NaN,82152796,"POLYGON ((4635715.588 3011706.509, 4635715.745...",335.475716,84.197815
...,...,...,...,...,...,...,...,...
252246,252246,None,None,NaN,82306857,"POLYGON ((4642297.478 3007551.433, 4642299.869...",330.080277,78.879577
252634,252634,None,None,NaN,82307471,"POLYGON ((4638144.878 3000005.098, 4638150.972...",495.780258,100.736102
252680,252680,None,None,NaN,82307723,"POLYGON ((4651393.717 3010388.624, 4651399.433...",210.314770,83.086081
252688,252688,None,None,NaN,82307737,"POLYGON ((4647580.946 3009591.337, 4647600.37 ...",595.565187,98.290569


In [11]:
masked.geometry = masked.buffer(1e-3)

In [12]:
subgraph = contig.subgraph(masked.index)

In [13]:
merged = masked.dissolve(subgraph.component_labels).explode()

In [14]:
fixed = pd.concat([buildings[~relevant], merged], ignore_index=True)

In [15]:
fixed.shape

(447728, 8)

In [16]:
fixed

,index,currentUse,buildingNature,heightAboveGround,iid,geometry,area,perimeter
0,0,None,None,NaN,80677517,"POLYGON ((4616718.204 3026081.581, 4616720.974...",137.301434,49.637208
1,1,None,None,NaN,80677518,"POLYGON ((4617480.926 3025927.323, 4617486.098...",130.923970,55.075112
2,2,None,None,NaN,80677519,"POLYGON ((4617331.885 3025855.863, 4617331.941...",91.956941,43.458137
3,3,None,None,NaN,80677520,"POLYGON ((4617495.899 3025921.454, 4617499.35 ...",69.749808,37.296935
4,4,None,None,NaN,80677521,"POLYGON ((4617561.862 3026075.057, 4617564.522...",95.799867,43.110199
...,...,...,...,...,...,...,...,...
447723,252246,None,None,NaN,82306857,"POLYGON ((4642297.478 3007551.432, 4642297.478...",330.080277,78.879577
447724,252634,None,None,NaN,82307471,"POLYGON ((4638144.878 3000005.097, 4638144.878...",495.780258,100.736102
447725,252680,None,None,NaN,82307723,"POLYGON ((4651393.716 3010388.624, 4651393.716...",210.314770,83.086081
447726,252688,None,None,NaN,82307737,"POLYGON ((4647580.945 3009591.336, 4647580.945...",595.565187,98.290569


In [18]:
import lonboard

In [ ]:
lonboard.viz(fixed)

In [21]:
fixed.drop(columns=["index", 'area', 'perimeter']).to_parquet("/data/uscuni-ulce/processed_data/simplified_buildings/buildings_69333.parquet")